# Data Preparation

## LLM Modell

Die Klasse definiert zunächst zentrale Parameter wie Host, Port, Timeout und Standardoptionen für die Modellgenerierung (z. B. Temperatur, Top-k, Stopbedingungen). Über die Methode models() kann geprüft werden, welche Modelle auf dem Server verfügbar sind. Mit pull_model() lassen sich Modelle bei Bedarf vom zentralen Ollama-Repository herunterladen und bereitstellen.

Für die Textgenerierung stehen zwei Hauptfunktionen zur Verfügung:

- completion() sendet einfache Prompts an das Modell.

- chat() ermöglicht einen Austausch im Format eines Chatverlaufs mit mehreren Rollen (z. B. Nutzer und Modell).

Beide Methoden nutzen intern api_request(), die die Kommunikation mit dem Ollama-Server übernimmt und dafür sorgt, dass Anfragen entweder als Streaming (Debugzwecken) oder klassisch per JSON beantwortet werden.

Die Antwortverarbeitung erfolgt robust durch zwei weitere Methoden:

- secure_text_response() extrahiert das Ergebnis aus der JSON-Antwort, berechnet die Dauer und die Anzahl generierter Tokens.

- secure_json_response() prüft zusätzlich, ob die Antwort im JSON- oder Markdown-Format vorliegt und verarbeitet ggf. auch eingebettete <think>-Blöcke, falls das Modell seine Begründung mitschickt.

Um sicherzustellen, dass fehlerhafte Kodierungen oder Zeichenfolgen nicht zu Problemen führen, wurde fix_invalid_escapes() ergänzt, die mithilfe der ftfy-Bibliothek Textprobleme korrigiert.

Diese API-Klasse stellt damit eine flexible Schnittstelle dar, um eigene Texte (wie Aussagen aus Nachhaltigkeitsberichten) effizient mit lokalen Sprachmodellen zu analysieren.

In [11]:
import requests
import json
import ftfy
import os
from bs4 import BeautifulSoup
from ddgs import DDGS
import time

In [12]:
class OllamaApi:

    HOST = "https://f2ki-h100-1.f2.htw-berlin.de"
    PORT = 11435

    TIMEOUT = 120
    STREAM_RESPONSE = False

    THINKING = False

    FALSE_RETURN = {"result": None, "time": 0, "token": 0, "info": {}}

    DEFAULT_OPTIONS = {
        "num_ctx": 2048,        # Default: 2048
        "repeat_last_n": 64,    # Default: 64, 0 = disabled, -1 = num_ctx
        "repeat_penalty": 1.1,  # Default: 1.1
        "temperature": 0.8,     # Default: 0.8
        "seed": 0,              # Default: 0
        "stop": [],             # No default
        "num_predict": -1,      # Default: -1, infinite generation
        "top_k": 40,            # Default: 40
        "top_p": 0.9,           # Default: 0.9
        "min_p": 0.0            # Default: 0.0
    }

    @staticmethod
    def fix_invalid_escapes(s):
        if not isinstance(s, str):
            return s

        try:
            s_re = ftfy.fix_text(s)
            if s_re != s:
                s = s_re
        except Exception as e:
            print(f"Encoding failed for text: {e}")

        return s

    @classmethod
    def models(cls):
        url = f"{cls.HOST}:{cls.PORT}/api/tags"
        headers = {
            "accept": "application/json",
        }
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Request failed with status {response.status_code}: {response.text}")
            return False
        else:
            try:
                json_data = response.json()
                return json_data.get("models", [])

            except json.JSONDecodeError as e:
                print(f"Failed to parse JSON: {e}")
                return False

    @classmethod
    def pull_model(cls, name:str, tag:str):
        url = f"{cls.HOST}:{cls.PORT}/api/pull"
        headers = {
            "Content-Type": "application/json",
            "accept": "application/json"
        }
        payload = {
            "model": f"{name}:{tag}"
        }
        try:
            response = requests.post(url, headers=headers, json=payload, stream=True, timeout=cls.TIMEOUT)
            if response.status_code != 200:
                print(f"Pull request failed with status {response.status_code}: {response.text}")
                return False

            for line in response.iter_lines():
                if line:
                    try:
                        progress = json.loads(line)
                        if 'status' in progress:
                            print(f"Pulling model: {progress['status']}")
                        if progress['status'] == "success":
                            return True
                    except json.JSONDecodeError:
                        continue

            return True
        except Exception as e:
            print(f"Failed to pull model: {e}")
            return False

    @classmethod
    def completion(cls, prompt:str, model="phi4:latest", schema=None, options=None):
        payload = {
            "model": model,
            "prompt" : prompt,
            "options": {
                **cls.DEFAULT_OPTIONS,
                **options
            } if options is not None else cls.DEFAULT_OPTIONS,
        }
        if schema is not None:
            payload["format"] = schema

        return cls.api_request(payload, force_json=False if schema is None else True)

    @classmethod
    def chat(cls, chat, model="phi4:latest", schema=None, options=None):
        payload = {
            "model": model,
            "messages": chat,
            "options": {
                **cls.DEFAULT_OPTIONS,
                **options
            } if options is not None else cls.DEFAULT_OPTIONS,
        }
        if schema is not None:
            payload["format"] = schema

        return cls.api_request(payload, force_json=False if schema is None else True)

    @classmethod
    def api_request(cls, payload, force_json:bool):
        if "messages" in payload:
            # Chat Request
            url = f"{cls.HOST}:{cls.PORT}/api/chat"
        else:
            # Completion Request
            url = f"{cls.HOST}:{cls.PORT}/api/generate"

        headers = {
            "Content-Type": "application/json",
            "accept": "application/json"
        }

        payload = {
            **payload,
            "think": cls.THINKING,
            "stream": cls.STREAM_RESPONSE,
            "keep_alive": "5m"
        }

        try:
            response = requests.post(url, headers=headers, json=payload, stream=cls.STREAM_RESPONSE, timeout=cls.TIMEOUT)

            if cls.STREAM_RESPONSE:
                for line in response.iter_lines(decode_unicode=True):
                    try:
                        chunk = json.loads(line)

                        if 'message' in chunk and 'content' in chunk['message']:
                            content = chunk['message']['content']
                            print(content, end='', flush=True)

                        if 'done' in chunk and chunk['done']:
                            break
                    except json.JSONDecodeError as e:
                        print(f"ERROR: Failed to decode JSON during streaming: {e}")
                return {**cls.FALSE_RETURN, "info": {"error": 'Streaming mode does not retreive a value'}}
            else:
                return cls.secure_json_response(response) if force_json else cls.secure_text_response(response)

        except requests.exceptions.Timeout:
            print(f"ERROR: The request took to long. Adjust the timeout ({cls.TIMEOUT}) as needed")
            return {**cls.FALSE_RETURN, "info": {"error": f"Request timeout ({cls.TIMEOUT}) reached"}}
        except Exception as e:
            print(f"ERROR: Request exception: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": f"Request exception: {e}"}}


    @classmethod
    def secure_json_response(cls, response):

        text_response = cls.secure_text_response(response)

        if text_response.get("result") is None:
            return text_response

        message = str(text_response.get("result"))

        markdown_response = False
        thinking_block = False

        if message.strip().startswith("<think>"):
            print('WARN: Model returned <think> reasoning block before JSON')
            message = re.sub(r"^\s*<think>.*?</think>\s*", "", message, flags=re.DOTALL).strip()
            thinking_block = True

        match = re.search(r'```json(.*?)```', message, re.DOTALL)
        if match:
            # alles außer dem Inhalt von „```json“ bis „```“ entfernen
            print('WARN: Model returned markdown instead of only JSON')
            message = match.group(1).strip()
            markdown_response = True

        try:
            # JSON parsen
            result = json.loads(message)

            # Überschreiben des Textergebnisses mit JSON dict
            text_response["result"] = dict(result)
            text_response["info"] = {
                "thinking": thinking_block,
                "markdown": markdown_response
            }
            return text_response

        except json.JSONDecodeError as e:
            print(f"ERROR: Failed to decode JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": 'JSON decode error on the model\'s response'}}

        except Exception as e:
            print(f"ERROR: Failed to parse JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": str(e)}}

    @classmethod
    def secure_text_response(cls, response):

        try:
            parsed_json = response.json()

            if response.status_code != 200:
                err_msg = parsed_json.get('error', 'Unknown error')
                print(f"ERROR: Request failed with status {response.status_code}: {err_msg}")
                return {**cls.FALSE_RETURN, "info":{"error":err_msg}}

            if 'done' not in parsed_json or parsed_json.get('done') is False:
                print("ERROR: Response has returned but Model didn't complete the answer")
                return {**cls.FALSE_RETURN, "info": {"error": 'Incomplete answer'}}

            # LLM Chat Rückgabe als String
            message = parsed_json.get('message').get('content') if "message" in parsed_json else parsed_json.get('response')

            microseconds_elapsed = parsed_json.get('total_duration')
            seconds_elapsed = round(microseconds_elapsed / 1000000000, 3)
            token_count = parsed_json.get('eval_count')

            return {
                "result": cls.fix_invalid_escapes(message),
                "time": float(seconds_elapsed),
                "token": int(token_count),
                "info": {}
            }

        except json.JSONDecodeError as e:
            print(f"ERROR: Failed to decode JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": 'JSON decode error on the ollama server\'s response'}}

        except Exception as e:
            print(f"ERROR: Failed to parse JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": str(e)}}

## Vorverarbeitung

In [13]:
import fitz
import re
import unicodedata

Text aus PDF-Dateien extrahieren

In [14]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text += page.get_text()
        doc.close()
    except Exception as e:
        print(f"Fehler beim Lesen von {pdf_path}: {e}")
    return text

Bereinigung des Texts von zusätzlichen Leerzeichen, Bindestrichen,...

In [15]:
def clean_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.replace('\xad', '')
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
    text = text.replace("\n", " ")
    text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != 'C')
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\s+([.,;:!?])', r'\1', text)
    return text.strip()

Aufteilung des Textes in Sätze

In [16]:
def split_sentences(text):
    protected = {
        r'\bz\. *B\.': 'z.B.',
        r'\bu\. *a\.': 'u.a.',
        r'\bu\. *Ä\.': 'u.Ä.',
        r'\bd\. *h\.': 'd.h.',
        r'\bu\. *s\. *w\.': 'usw.',
    }
    for pattern, replacement in protected.items():
        text = re.sub(pattern, replacement, text)
    sentences = re.split(r'(?<=[.!?])\s+(?=[A-ZÄÖÜ])', text)
    return [s.strip() for s in sentences if s.strip()]

Gruppierung der Sätze in Chunks

In [17]:
def group_sentences(sentences, max_length=500):
    chunks, current = [], ""
    for sentence in sentences:
        if len(current) + len(sentence) + 1 <= max_length:
            current += " " + sentence if current else sentence
        else:
            chunks.append(current.strip())
            current = sentence
    if current:
        chunks.append(current.strip())
    return chunks

Kombination der Aufteilung von Sätzen und Chunks

In [18]:
def split_and_chunk_text(text, max_length=500):
    sentences = split_sentences(text)
    return group_sentences(sentences, max_length)

## Filterung von Nachhaltigkeitsversprechen mit LLM

Im folgenden Abschnitt wird der vollständige Workflow zur Analyse von PDF-Dokumenten (z. B. Nachhaltigkeits- oder Geschäftsberichte) beschrieben. Ziel ist es, mithilfe eines großen Sprachmodells relevante Nachhaltigkeitsversprechen aus dem Text zu extrahieren. Der Code ist in mehrere Teilfunktionen gegliedert, um Wiederverwendbarkeit und Übersichtlichkeit zu gewährleisten.

Modell und Prompt wählen

In [19]:
# Modell wählen
model_name = "llama3.1:70b"
print(f"-------\nUsing model: {model_name}")

# Ordnerpfad mit PDFs
pdf_folder_path = "./reports"

# Prompt-Vorlage
prompt_template = """
Extrahiere bitte alle Nachhaltigkeitsversprechen, die ein konkretes Ziel verfolgen, aus folgendem Text. Gib nur den relevanten Text zurück, keine Einleitung oder sonstigen Text:

{text}
"""

-------
Using model: llama3.1:70b


Analyse der PDF-Datei

In [20]:
# PDF-Dateien durchgehen
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".pdf"):
        full_path = os.path.join(pdf_folder_path, filename)
        print(f"\n---\nVerarbeite Datei: {filename}")

        # Text extrahieren und bereinigen
        raw_text = extract_text_from_pdf(full_path)
        if not raw_text.strip():
            print("PDF enthält keinen extrahierbaren Text.")
            continue

        cleaned_text = clean_text(raw_text)
        chunks = split_and_chunk_text(cleaned_text, max_length=500)
        print(f"PDF in {len(chunks)} Chunks aufgeteilt.")

        all_results = []

        for i, chunk in enumerate(chunks):
            print(f"\n→ Chunk {i+1}/{len(chunks)} wird analysiert...")
            prompt = prompt_template.format(text=chunk)
            try:
                result = OllamaApi.completion(prompt, model=model_name)
                extracted = result.get("result", "").strip()
                if extracted:
                    all_results.append(extracted)
            except Exception as e:
                print(f"Fehler bei der Modellabfrage (Chunk {i+1}): {e}")

        # Ergebnisse anzeigen
        print(f"\n### Extrahierte Nachhaltigkeitsversprechen aus {filename}:\n")
        for i, res in enumerate(all_results):
            print(f"--- Chunk {i+1} ---\n{res}\n")



---
Verarbeite Datei: 2023_Volkswagen_Group_Nachhaltigkeitsbericht.pdf
PDF in 1164 Chunks aufgeteilt.

→ Chunk 1/1164 wird analysiert...

→ Chunk 2/1164 wird analysiert...

→ Chunk 3/1164 wird analysiert...

→ Chunk 4/1164 wird analysiert...

→ Chunk 5/1164 wird analysiert...

→ Chunk 6/1164 wird analysiert...

→ Chunk 7/1164 wird analysiert...

→ Chunk 8/1164 wird analysiert...

→ Chunk 9/1164 wird analysiert...

→ Chunk 10/1164 wird analysiert...

→ Chunk 11/1164 wird analysiert...

→ Chunk 12/1164 wird analysiert...

→ Chunk 13/1164 wird analysiert...

→ Chunk 14/1164 wird analysiert...

→ Chunk 15/1164 wird analysiert...

→ Chunk 16/1164 wird analysiert...

→ Chunk 17/1164 wird analysiert...

→ Chunk 18/1164 wird analysiert...

→ Chunk 19/1164 wird analysiert...

→ Chunk 20/1164 wird analysiert...

→ Chunk 21/1164 wird analysiert...

→ Chunk 22/1164 wird analysiert...

→ Chunk 23/1164 wird analysiert...

→ Chunk 24/1164 wird analysiert...

→ Chunk 25/1164 wird analysiert...

→ Chu

Filterfunktion zur Erkennung positiver Nachhaltigkeitsversprechen in Texten

In [21]:
def is_promise(text):
    negativ_phrase = [
        "keine", "nicht enthalten", "nicht vorhanden", "leider", "keine Angaben",
        "keine relevanten", "keine Informationen", "keine konkreten", "keine expliziten",
        "keine spezifischen Ziele", "nur Begriffserklärungen", "keine Nachhaltigkeitsversprechen",
    ]
    return not any(phrase.lower() in text.lower() for phrase in negativ_phrase)

promises = [res for res in all_results if is_promise(res)]

Speichern des Codes in einer CSV-Datei

In [22]:
# Ergebnisse anzeigen
print(f"\n### Extrahierte Nachhaltigkeitsversprechen aus {filename}:\n")
for i, res in enumerate(promises):
    print(f"--- Chunk {i+1} ---\n{res}\n")

# Speichern extrahierter Inhalte zur Weiterverarbeitung
with open(f"extracted_{filename}.txt", "w", encoding="utf-8") as f:
    for res in promises:
        f.write(res + "\n")

# Speichern in einer Liste für spätere Analyse mit KeyBERT / BERTopic
if 'filtered_chunks' not in globals():
    filtered_chunks = []
filtered_chunks.extend(promises)



### Extrahierte Nachhaltigkeitsversprechen aus 2023_Volkswagen_Group_Nachhaltigkeitsbericht.pdf:

--- Chunk 1 ---
Ich bin bereit, dir zu helfen! Bitte füge den Text hinzu, aus dem ich die Nachhaltigkeitsversprechen extrahieren soll.

--- Chunk 2 ---
* "Wir verfolgen das Ziel, unsere Leistung in ESG-Ratings und -Rankings des Kapitalmarkts zu verbessern, um die Investitionsfähigkeit zu steigern und Kapitalkosten zu optimieren."
* "Wir beabsichtigen, Fortschritte bei Transparenz und Risikominderung in den Rohstofflieferketten (jährlich) im Responsible Raw Materials Report darzustellen."
* "Wir streben die Umsetzung nachhaltiger Finanzierungsstrategien an, die jährlich im Green Finance Report dargestellt werden sollen."
* "Wir verpflichten uns zur Umsetzung der zehn UN-Global-Compact-Prinzipien, die jährlich berichtet wird."

--- Chunk 3 ---
ein weltweit führender Anbieter nachhaltiger Mobilität zu sein

--- Chunk 4 ---
Bis 2030 soll der CO2-Fußabdruck des gesamten Handelsnetzes um mindes

## KeyBERT zur Extraktion der relevantesten Aussagen

In diesem Schritt wird mithilfe von KeyBERT und einem vortrainierten SentenceTransformer-Modell (MiniLM) aus jedem Textabschnitt (chunk) eine Liste der wichtigsten Schlüsselbegriffe (Keywords) extrahiert. Diese Keywords dienen als Grundlage für die semantische Analyse und spätere Filterung von relevanten Nachhaltigkeitsaussagen.

In [23]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic

def preprocess_text(text):
    return text.strip()

# Preprocessing der Inhalte
preprocessed_filtered_chunks = [preprocess_text(chunk) for chunk in filtered_chunks]

# KeyBERT Keywords
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
kw_model = KeyBERT(model=model)

print("\nKeyBERT Keyword-Analyse:")
for i, chunk in enumerate(preprocessed_filtered_chunks[:5]):
    keywords = kw_model.extract_keywords(chunk, keyphrase_ngram_range=(1, 3), stop_words=None, top_n=5)
    print(f"Chunk {i+1} Keywords: {keywords}\n")

# BERTopic Topic Modeling
embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
topic_model = BERTopic(language="german", embedding_model=embedding_model)

topics, probs = topic_model.fit_transform(preprocessed_filtered_chunks)

print("\nBERTopic Topics:")
print(topic_model.get_topic_info())

print("\nBeispiel Topic 1:")
print(topic_model.get_topic(1))


c:\Dokumente\Uni Master\SoSe25\Unternehmenssoftware\Projekt\Greenwashing_Detector\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



KeyBERT Keyword-Analyse:
Chunk 1 Keywords: [('ich die nachhaltigkeitsversprechen', 0.7322), ('ich bin bereit', 0.7167), ('ich bin', 0.6332), ('dem ich', 0.6062), ('ich die', 0.5989)]

Chunk 2 Keywords: [('transparenz und risikominderung', 0.5728), ('umsetzung nachhaltiger finanzierungsstrategien', 0.5537), ('steigern und kapitalkosten', 0.5174), ('bei transparenz und', 0.5165), ('verpflichten uns zur', 0.5145)]

Chunk 3 Keywords: [('führender anbieter nachhaltiger', 0.7492), ('anbieter nachhaltiger mobilität', 0.7213), ('nachhaltiger mobilität zu', 0.7186), ('ein weltweit führender', 0.7162), ('ein weltweit', 0.6696)]

Chunk 4 Keywords: [('der co2 fußabdruck', 0.6636), ('co2 fußabdruck des', 0.6554), ('gesenkt werden zugang', 0.6191), ('fußabdruck des gesamten', 0.6078), ('soll der co2', 0.5939)]

Chunk 5 Keywords: [('co2 wurden seit', 0.6184), ('den aluminiumclosed loop', 0.5576), ('000 co2 wurden', 0.5533), ('co2 wurden', 0.5466), ('den aluminiumclosed', 0.5288)]


BERTopic Topics:


## Artikel Vorverarbeitung

Der Artikel wird aus dem Internet gescrapt und anschließend vorverarbeitet.

In [24]:
from newspaper import Article

url = "https://nachhaltigkeit-wirtschaft.de/nachhaltigkeitsbericht-volkswagen-2022-wie-wir-den-wandel-zu-nachhaltiger-mobilitaet-vorantreiben/"
article = Article(url, language='de')
article.download()
article.parse()
article_text = article.text

Vorverarbeitung des Artikeltexts

In [25]:
cleaned_article_text = clean_text(article_text)
article_sentences = split_sentences(cleaned_article_text)
print(f"Anzahl Artikel-Sätze: {len(article_sentences)}")

Anzahl Artikel-Sätze: 69


In [26]:
preprocessed_article_chunks = [preprocess_text(chunk) for chunk in article_sentences]

print(f"Beispiel vorverarbeiteter Chunk:\n{preprocessed_article_chunks[0][:200]}")

Beispiel vorverarbeiteter Chunk:
Nachhaltigkeitsbericht Volkswagen 2022: Strategische Bedeutung für den Konzern Nachhaltigkeitsbericht Volkswagen 2022: Strategische Bedeutung für den Konzern Werbung Der Nachhaltigkeitsbericht Volkswa


Themenmodellierung mit BERTopic für kleine Textdatensätze

In [27]:
from umap import UMAP
import hdbscan

embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

umap_model = UMAP(n_neighbors=3, n_components=5, metric='cosine')
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=2, metric='euclidean', prediction_data=True)

topic_model = BERTopic(
    language="german",
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model
)

topics, probs = topic_model.fit_transform(preprocessed_article_chunks)

topic_info = topic_model.get_topic_info()
print(topic_info)

print("\nBeispiel Topic 1:")
print(topic_model.get_topic(1))

   Topic  Count                                               Name  \
0     -1      1                  -1_profitieren_marke_kunden_einer   
1      0     25                               0_und_werden_die_der   
2      1     11       1_volkswagen_2022_nachhaltigkeitsbericht_bei   
3      2      8                  2_wandel_mobilität_volkswagen_auf   
4      3      8      3_der_innovationen_nachhaltige_transformation   
5      4      6                    4_leitprinzip_die_standards_als   
6      5      4                              5_konzern_ein_der_und   
7      6      4  6_investoren_herausforderungen_aus_berichterst...   
8      7      2           7_produkt_fokus_geschäftsentwicklung_des   

                                      Representation  \
0  [profitieren, marke, kunden, einer, unterstütz...   
1  [und, werden, die, der, regelmäßig, als, von, ...   
2  [volkswagen, 2022, nachhaltigkeitsbericht, bei...   
3  [wandel, mobilität, volkswagen, auf, so, den, ...   
4  [der, innovation

# Modeling

## Semantische Analyse der Versprechen aus dem Zeitungsartikel und dem Nachhaltigkeitsbericht

Der Code berechnet Embeddings für zwei Textsammlungen (Artikel- und Bericht-Chunks) mit einem vortrainierten Modell und bestimmt dann für jeden Artikel-Chunk die Top 3 ähnlichsten Bericht-Chunks anhand der Kosinus-Ähnlichkeit. Die ähnlichsten Textausschnitte werden mit ihren Ähnlichkeitsscores ausgegeben, um vergleichbare Inhalte zwischen den Textgruppen zu identifizieren.

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

all_report_embeddings = embedding_model.encode(preprocessed_filtered_chunks, convert_to_tensor=True)
all_article_embeddings = embedding_model.encode(preprocessed_article_chunks, convert_to_tensor=True)

similarity_matrix = cosine_similarity(all_article_embeddings, all_report_embeddings)

top_k = 3

for i, article_chunk in enumerate(preprocessed_article_chunks):
    similarities = similarity_matrix[i]
    top_indices = np.argsort(similarities)[::-1][:top_k]
    
    print(f"\nArtikel-Chunk {i+1}:\n{article_sentences[i][:300]}...\n")

    for rank, idx in enumerate(top_indices):
        print(f"Ähnlichster Bericht-Chunk {rank+1} (Score: {similarities[idx]:.2f}):")
        print(filtered_chunks[idx][:300])
        print("-" * 80)


Artikel-Chunk 1:
Nachhaltigkeitsbericht Volkswagen 2022: Strategische Bedeutung für den Konzern Nachhaltigkeitsbericht Volkswagen 2022: Strategische Bedeutung für den Konzern Werbung Der Nachhaltigkeitsbericht Volkswagen 2022 markiert einen Wendepunkt in der Unternehmensstrategie des Konzerns....

Ähnlichster Bericht-Chunk 1 (Score: 0.81):
leisten...einen wichtigen Beitrag für mehr Nachhaltigkeit im Volkswagen Konzern
--------------------------------------------------------------------------------
Ähnlichster Bericht-Chunk 2 (Score: 0.77):
Der Volkswagen Konzern wendet sich entschieden gegen Zwangs- und Kinderarbeit im Zusammenhang mit seinen Geschäftsaktivitäten.
--------------------------------------------------------------------------------
Ähnlichster Bericht-Chunk 3 (Score: 0.74):
Beschäftigungssicherung bis 2029 in der Volkswagen AG
--------------------------------------------------------------------------------

Artikel-Chunk 2:
Nachhaltigkeit ist längst kein Randthema mehr, so

Speichern der Daten

In [34]:
similarity_results = []

for i, article_chunk in enumerate(preprocessed_article_chunks):
    similarities = similarity_matrix[i]
    top_indices = np.argsort(similarities)[::-1][:3]
    
    matches = []
    for idx in top_indices:
        matches.append({
            "bericht_text": filtered_chunks[idx],
            "similarity": float(similarities[idx])
        })
    
    similarity_results.append({
        "artikel_chunk": article_chunk,
        "top_matches": matches
    })

# Speichern
with open("similarity_data.json", "w", encoding="utf-8") as f:
    json.dump(similarity_results, f, ensure_ascii=False, indent=2)

## Zero-Shot-Klassifikation von Nachhaltigkeitsaussagen mit BART aus den nachhaltigkeitsberichten

Der Code nutzt ein vortrainiertes Zero-Shot-Klassifikationsmodell („facebook/bart-large-mnli“), um Textabschnitte (Chunks) aus Versprechen im Nachhaltigkeitskontext automatisch in vorgegebene Kategorien wie „konkrete Maßnahme“, „vage Behauptung“ oder „fragwürdige Nachhaltigkeitsaussage“ einzuordnen. Dabei wird für jeden Text die wahrscheinlichste Kategorie bestimmt, wobei bei unsicherer Klassifikation (niedriger Score oder kleine Score-Differenz) eine „unsicher“-Kennzeichnung vergeben wird.

In [29]:
from transformers import pipeline

# Modell laden
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Kategorien, auf die klassifiziert werden soll
labels = [
    "konkrete Maßnahme",
    "vage Behauptung",
    "fragwürdige Nachhaltigkeitsaussage"
]

classification_results = []
filtered_chunks = list(promises)
filtered_chunks = [chunk for chunk in filtered_chunks if not chunk.lower().startswith("ich bin bereit") and "füge den text hinzu" not in chunk.lower()]

for chunk in filtered_chunks:
    result = classifier(
        chunk,
        candidate_labels=labels,
        hypothesis_template="Diese Aussage stellt eine {} im Kontext von Nachhaltigkeit dar."
    )
    
    # wenn der höchste Score unter 0.5 ist, dann ist die Einordnung unsicher
    top_label = result["labels"][0]
    top_score = result["scores"][0]
    second_score = result["scores"][1]
    score_diff = top_score - second_score
    
    if top_score < 0.5 or score_diff < 0.1:
        label = "unsicher"
    else:
        label = top_label
    
    classification_results.append({
        "text": chunk,
        "label": label,
        "score": top_score,
        "score_diff": score_diff,
        "all_labels": result["labels"],
        "all_scores": result["scores"]
    })

Device set to use cpu


In [30]:
import pandas as pd

df_classified = pd.DataFrame(classification_results)
df_sorted = df_classified.sort_values(by="score", ascending=False)

df_greenwashing = df_sorted[df_sorted["label"] == "fragwürdige Nachhaltigkeitsaussage"]
print(df_greenwashing.head(10)[["text", "label", "score"]])

                                                  text  \
200  Es gibt im gegebenen Text kein explizites Nach...   
187  Es gibt kein Nachhaltigkeitsversprechen mit ei...   
191  Der Text enthält kein explizites Nachhaltigkei...   

                                  label     score  
200  fragwürdige Nachhaltigkeitsaussage  0.649570  
187  fragwürdige Nachhaltigkeitsaussage  0.623605  
191  fragwürdige Nachhaltigkeitsaussage  0.528854  


In [31]:
df_classified["label"].value_counts()

label
unsicher                              154
konkrete Maßnahme                     119
vage Behauptung                        23
fragwürdige Nachhaltigkeitsaussage      3
Name: count, dtype: int64

In [32]:
df = pd.DataFrame(classification_results)
df["text_short"] = df["text"].str.slice(0, 150)
df.to_csv("greenwashing_classification.csv", index=False)

In [33]:
for label in df['label'].unique():
    print(f"\n{label} (Beispiele):")
    beispiele = df[df["label"] == label].sort_values(by="score", ascending=False).head(3)
    for i, row in beispiele.iterrows():
        print(f"   {i+1}. {row['text_short']}... (Score: {row['score']:.2f})")


konkrete Maßnahme (Beispiele):
   106. Als Mitglied in der Arbeitsgruppe des Mobility Model (MoMo) der International Energy Agency (IEA) nutzen wir beispielsweise sogenannte IEA-ETP-Szenari... (Score: 0.79)
   43. Entwicklung einheitlicher Bewertungsstandards für die ImpactBewertung sowie die finanzielle Bilanzierung von Nachhaltigkeitswirkungen.... (Score: 0.77)
   227. Die folgenden konkreten Nachhaltigkeitsversprechen wurden gefunden:

* Reduzierung des Energiebedarfs um etwa 3.250 MWh pro Jahr
* Reduzierung des CO2... (Score: 0.76)

vage Behauptung (Beispiele):
   35. Ein nachhaltiger Konzern... (Score: 0.66)
   14. Wir wollen nachhaltige Mobilität für Generationen ermöglichen.... (Score: 0.66)
   5. Unser Ziel ist es, die Natur und die Gesellschaft positiv mitzugestal ten.... (Score: 0.64)

unsicher (Beispiele):
   21. Eine zukunftsorientierte Mechatronikplattform soll (...) die nachhaltige Wettbewerbsfähigkeit bei Volkswagen bilden.... (Score: 0.53)
   10. Veränderungen steuern 

# Evaluation